_PRELIMINARIES_

In [1]:
import pandas as pd

_SET GLOBAL PARAMETERS_

In [2]:
EDO = 12
TITLE = 'Moz157_8beat_onset_mm40-52'
UPPERLIMIT = EDO//2 + 1

master_df = pd.read_csv(f'C:\\Users\\jennd\\Desktop\\Master_DFs\\{TITLE}_masterDF.csv')

_FUNCTIONS_

In [3]:
def fix_index(phaseIndex):
    if abs(phaseIndex) <= 180:
        return abs(phaseIndex)
    else:
        return abs(360 - abs(phaseIndex))

In [4]:
def find_phase_index_ratings(df):
    working_dict = {'components' : [], 'rating' : []}
    for x in range(1, UPPERLIMIT):
        for y in range(x+1, UPPERLIMIT):
            for z in range(1, UPPERLIMIT):
                if z != x and z != y:
                    working_dict['components'].append(str([x, y, z]))
                    phase_index_list = (abs(df[f'f{x} Phase'] + df[f'f{y} Phase'] - df[f'f{z} Phase'])).values.tolist()
                    norm = [fix_index(a) for a in phase_index_list]
                    working_dict['rating'].append(sum(norm) / len(norm))
    phase_index_ratings_df = pd.DataFrame(working_dict)
    return phase_index_ratings_df

In [5]:
def extract_phases(df):
    phase_key_list = [f'f{i} Phase' for i in range(1, UPPERLIMIT)]
    phase_df = df[phase_key_list].copy()
    new_column_names = {f'f{i} Phase' : f'f{i}' for i in range(1, UPPERLIMIT)}
    phase_df.columns = [f'f{i}' for i in range(1, UPPERLIMIT)]
    return phase_df

In [6]:
def expand_phase_df(df):
    for x in range(1, UPPERLIMIT):
        for y in range(x+1, UPPERLIMIT):
            df[f'f{x}+f{y}'] = df[f'f{x}'] + df[f'f{y}']
    return df

In [7]:
def save_correlations_csv(df):
    df.to_csv(f'C:\\Users\\jennd\\Desktop\\ExpandedPhaseCorrelations\\{TITLE}.csv')

_DO WORK_

In [8]:
# master_df.head()

In [9]:
phase_df = extract_phases(df=master_df)

In [10]:
phase_df.head()

,f1,f2,f3,f4,f5,f6
0,-20,-29,113,13,83,0
1,9,-22,100,19,83,0
2,-10,-22,93,28,81,0
3,-30,-30,89,14,76,0
4,-19,-35,90,35,75,0


In [11]:
expanded_phases_df = expand_phase_df(df=phase_df)

In [12]:
expanded_phases_df.head()

,f1,f2,f3,f4,f5,f6,f1+f2,f1+f3,f1+f4,f1+f5,...,f2+f3,f2+f4,f2+f5,f2+f6,f3+f4,f3+f5,f3+f6,f4+f5,f4+f6,f5+f6
0,-20,-29,113,13,83,0,-49,93,-7,63,...,84,-16,54,-29,126,196,113,96,13,83
1,9,-22,100,19,83,0,-13,109,28,92,...,78,-3,61,-22,119,183,100,102,19,83
2,-10,-22,93,28,81,0,-32,83,18,71,...,71,6,59,-22,121,174,93,109,28,81
3,-30,-30,89,14,76,0,-60,59,-16,46,...,59,-16,46,-30,103,165,89,90,14,76
4,-19,-35,90,35,75,0,-54,71,16,56,...,55,0,40,-35,125,165,90,110,35,75


In [13]:
correlated_df = expanded_phases_df.corr()

In [14]:
correlated_df.head()

,f1,f2,f3,f4,f5,f6,f1+f2,f1+f3,f1+f4,f1+f5,...,f2+f3,f2+f4,f2+f5,f2+f6,f3+f4,f3+f5,f3+f6,f4+f5,f4+f6,f5+f6
f1,1.000000,0.277540,0.012509,0.023594,0.335509,0.882838,0.955038,0.941085,0.826192,0.995358,...,0.453169,0.222039,0.312121,0.869570,0.021727,0.118569,0.897808,0.083411,0.874791,0.884238
f2,0.277540,1.000000,-0.798433,-0.688666,0.629646,0.411806,0.549896,-0.010012,-0.162484,0.331067,...,0.293556,-0.236683,0.981375,0.498010,-0.796997,-0.652987,0.326170,-0.658149,0.257915,0.428090
f3,0.012509,-0.798433,1.000000,0.623278,-0.348758,-0.188787,-0.235527,0.349915,0.361414,-0.023610,...,0.341172,0.287362,-0.745489,-0.264390,0.825554,0.955808,-0.076520,0.633344,-0.052238,-0.198237
f4,0.023594,-0.688666,0.623278,1.000000,-0.698155,-0.039188,-0.192014,0.232894,0.582725,-0.048653,...,-0.085585,0.867473,-0.741253,-0.110381,0.955851,0.444920,0.031989,0.992527,0.175978,-0.061821
f5,0.335509,0.629646,-0.348758,-0.698155,1.000000,0.317172,0.486000,0.196374,-0.120708,0.424618,...,0.429301,-0.503502,0.767159,0.368649,-0.634870,-0.057815,0.281870,-0.605571,0.162650,0.346728


In [15]:
# save_correlations_csv(df=correlated_df)

In [16]:
phase_df['f1'].corr(phase_df['f2'])

0.2775396910565073

In [ ]:
min(haba_df['rating'])

In [ ]:
haba_df.loc[haba_df['rating'] == min(haba_df['rating'])]

In [ ]:
haba_df.loc[haba_df['rating'] < 80]